In [2]:
import cv2
import mediapipe as mp
import random
import time

# Configuraciones de MediaPipe para la detección de manos
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Inicializar la captura de video de la cámara
cap = cv2.VideoCapture(0)

# Variables para el juego
gestos = ["Piedra", "Papel", "Tijera"]
puntaje_usuario = 0
puntaje_maquina = 0
ultimo_tiempo = time.time()
gesto_maquina = random.choice(gestos)
puntaje_actualizado = False  # Bandera para controlar la actualización de puntaje
tiempo_cambio_gesto = 6  # Tiempo en segundos para cambiar el gesto de la máquina

# Iniciar la detección de manos con MediaPipe
with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
    while cap.isOpened():
        # Verificar si el juego ha alcanzado el número máximo de rondas
        ret, frame = cap.read()
        if not ret:
            break
        
        if puntaje_usuario == 3:
            break
        
        if puntaje_maquina == 3:
            break

        # Inicializar el resultado para evitar errores de referencia antes de ser definido
        resultado = ""

        # Convertir la imagen de BGR a RGB para MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        # Tiempo restante para el cambio del gesto de la máquina
        tiempo_restante = int(tiempo_cambio_gesto - (time.time() - ultimo_tiempo))

        # Variable para el gesto del usuario
        gesto_usuario = None

        # Si se detecta una mano en la imagen
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar las marcas en la mano
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Obtener la posición de las marcas relevantes en los dedos
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

                # Detectar el gesto del usuario
                if (index_tip < thumb_tip and middle_tip < thumb_tip and
                        ring_tip < thumb_tip and pinky_tip < thumb_tip):
                    gesto_usuario = "Papel"
                elif (index_tip < thumb_tip and index_tip < ring_tip and
                        middle_tip < thumb_tip and middle_tip < ring_tip and
                        ring_tip > index_tip and pinky_tip > index_tip):
                    gesto_usuario = "Tijera"
                elif (index_tip > thumb_tip and middle_tip > thumb_tip and
                        ring_tip > thumb_tip and pinky_tip > thumb_tip):
                    gesto_usuario = "Piedra"
                

        # Solo mostrar y comparar el gesto después de que el tiempo de cuenta regresiva llegue a 0
        if tiempo_restante == 0:
            # Actualizar el gesto de la máquina y resetear el tiempo
            if not puntaje_actualizado:
                gesto_maquina = random.choice(gestos)
                puntaje_actualizado = True  # Evitar que se actualice hasta la próxima ronda

                # Comparar el gesto del usuario con el de la máquina
                if gesto_usuario:
                    if gesto_usuario == gesto_maquina:
                        resultado = "Empate"
                    elif(gesto_usuario == "Piedra" and gesto_maquina == "Tijera") or \
                        (gesto_usuario == "Papel" and gesto_maquina == "Piedra") or \
                        (gesto_usuario == "Tijera" and gesto_maquina == "Papel"):
                        resultado = "Gana el usuario"
                        puntaje_usuario += 1
                    else:
                        resultado = "Gana la maquina"
                        puntaje_maquina += 1
                        
                # Resetear el tiempo para la próxima ronda
                ultimo_tiempo = time.time()
                puntaje_actualizado = False
        

        # Mostrar el marcador y la cuenta regresiva
        marcador_ancho = 200
        marcador_alto = 150

        # Rectángulo para el puntaje del usuario
        cv2.rectangle(frame, (50, 50), (50 + marcador_ancho, 50 + marcador_alto), (50, 50, 50), -1)
        cv2.putText(frame, "USUARIO", (60, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"{puntaje_usuario}", (120, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)

        # Rectángulo para el puntaje de la máquina
        cv2.rectangle(frame, (300, 50), (300 + marcador_ancho, 50 + marcador_alto), (50, 50, 50), -1)
        cv2.putText(frame, "MAQUINA", (310, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"{puntaje_maquina}", (370, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)

        # Rectángulo para el resultado del juego
        resultado_ancho = 400
        resultado_alto = 80
        cv2.rectangle(frame, (100, 250), (100 + resultado_ancho, 250 + resultado_alto), (0, 0, 0), -1)
        cv2.putText(frame, f"{resultado}", (120, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Mostrar el gesto de la máquina solo después de que el tiempo haya llegado a 0
        if tiempo_restante <= 0:
            cv2.putText(frame, f"Gesto maquina: {gesto_maquina}", (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Mostrar el gesto del usuario en pantalla
        if gesto_usuario:
            cv2.putText(frame, f"Gesto usuario: {gesto_usuario}", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        else:
            cv2.putText(frame, "Gesto usuario: No detectado", (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Mostrar la cuenta regresiva para el cambio de gesto si aún no se ha llegado a 0
        if tiempo_restante > 0:
            cv2.putText(frame, f"Cambio en: {tiempo_restante}s", (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Mostrar el video en tiempo real
        cv2.imshow('Piedra, Papel, Tijera - Deteccion de gestos', frame)

        # Salir al presionar la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1731601805.164590 3206297 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1731601805.185749 3210428 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731601805.192519 3210428 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
